In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error

In [2]:
index_names = ['unit_nr', 'time_cycles']
setting_names = ['setting_1', 'setting_2', 'setting_3']
sensor_names = ['s_{}'.format(i+1) for i in range(0,21)]
col_names = index_names + setting_names + sensor_names

In [3]:
train = pd.read_csv('train_FD001.txt', sep='\s+', header=None, 
                 names=col_names)
test = pd.read_csv('test_FD001.txt', sep='\s+', header=None, 
                 names=col_names)
y_test = pd.read_csv(('RUL_FD001.txt'), sep='\s+', header=None)

In [4]:
train[index_names].groupby('unit_nr').max().describe()

,time_cycles
count,100.000000
mean,206.310000
std,46.342749
min,128.000000
25%,177.000000
50%,199.000000
75%,229.250000
max,362.000000


In [5]:
def add_remaining_useful_life(df):
    # Get the total number of cycles for each unit
    grouped_by_unit = df.groupby(by="unit_nr")
    max_cycle = grouped_by_unit["time_cycles"].max()
    
    # Merge the max cycle back into the original frame
    result_frame = df.merge(max_cycle.to_frame(name='max_cycle'), left_on='unit_nr', right_index=True)
    
    # Calculate remaining useful life for each row
    remaining_useful_life = result_frame["max_cycle"] - result_frame["time_cycles"]
    result_frame["RUL"] = remaining_useful_life
    
    # drop max_cycle as it's no longer needed
    result_frame = result_frame.drop("max_cycle", axis=1)
    return result_frame

In [6]:
train = add_remaining_useful_life(train)
train[index_names+['RUL']]

,unit_nr,time_cycles,RUL
0,1,1,191
1,1,2,190
2,1,3,189
3,1,4,188
4,1,5,187
...,...,...,...
20626,100,196,4
20627,100,197,3
20628,100,198,2
20629,100,199,1


In [7]:
drop_sensors = ['s_1','s_5','s_16','s_18','s_19']  # s_6 and s_10 get the benefit of the doubt
drop_labels = index_names+setting_names+drop_sensors
remaining_sensors = ['s_2', 's_3', 's_4', 's_6', 's_7', 's_8', 's_9', 's_10',
       's_11', 's_12', 's_13', 's_14', 's_15', 's_17', 's_20', 's_21']

In [8]:
X_train = train.drop(drop_labels, axis=1)
y_train = X_train.pop('RUL')

In [9]:
X_test = test.groupby('unit_nr').last().reset_index().drop(drop_labels, axis=1)

In [10]:
X_test

,s_2,s_3,s_4,s_6,s_7,s_8,s_9,s_10,s_11,s_12,s_13,s_14,s_15,s_17,s_20,s_21
0,642.58,1581.22,1398.91,21.61,554.42,2388.08,9056.40,1.3,47.23,521.79,2388.06,8130.11,8.4024,393,38.81,23.3552
1,642.55,1586.59,1410.83,21.61,553.52,2388.10,9044.77,1.3,47.67,521.74,2388.09,8126.90,8.4505,391,38.81,23.2618
2,642.88,1589.75,1418.89,21.61,552.59,2388.16,9049.26,1.3,47.88,520.83,2388.14,8131.46,8.4119,395,38.93,23.2740
3,642.78,1594.53,1406.88,21.61,552.64,2388.13,9051.30,1.3,47.65,521.88,2388.11,8133.64,8.4634,395,38.58,23.2581
4,642.27,1589.94,1419.36,21.61,553.29,2388.10,9053.99,1.3,47.46,521.00,2388.15,8125.74,8.4362,394,38.75,23.4117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,642.30,1590.88,1397.94,21.61,553.99,2388.03,9062.41,1.3,47.14,522.30,2388.01,8148.24,8.4110,391,38.96,23.4606
96,642.59,1582.96,1410.92,21.61,554.05,2388.06,9076.36,1.3,47.38,521.58,2388.06,8155.48,8.4500,395,38.61,23.2953
97,642.68,1599.51,1415.47,21.61,553.44,2388.13,9062.34,1.3,47.66,521.53,2388.09,8146.39,8.4235,394,38.76,23.3608
98,642.00,1585.03,1397.98,21.61,554.75,2388.01,9067.16,1.3,47.26,521.82,2388.02,8150.38,8.4003,391,38.95,23.3595


In [11]:
# from tensorflow.keras.layers import BatchNormalization

In [12]:
ann = tf.keras.models.Sequential()

In [13]:
# ann.add(BatchNormalization())

In [14]:
ann.add(tf.keras.layers.Dense(units=500, activation='relu'))

In [15]:
ann.add(tf.keras.layers.Dense(units=100, activation='relu'))

In [16]:
ann.add(tf.keras.layers.Dense(units=50, activation='relu'))

In [17]:
# ann.add(tf.keras.layers.Dense(units=4, activation='relu'))

In [18]:
ann.add(tf.keras.layers.Dense(units=1))

In [19]:
ann.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [20]:
ann.fit(X_train, y_train ,batch_size = 32, epochs = 60,verbose=2)

Epoch 1/60
645/645 - 4s - loss: 8707.3145 - 4s/epoch - 6ms/step
Epoch 2/60
645/645 - 3s - loss: 5296.4678 - 3s/epoch - 4ms/step
Epoch 3/60
645/645 - 2s - loss: 5103.7944 - 2s/epoch - 4ms/step
Epoch 4/60
645/645 - 2s - loss: 4930.6001 - 2s/epoch - 3ms/step
Epoch 5/60
645/645 - 2s - loss: 4842.2241 - 2s/epoch - 3ms/step
Epoch 6/60
645/645 - 2s - loss: 4835.1885 - 2s/epoch - 3ms/step
Epoch 7/60
645/645 - 2s - loss: 4828.9077 - 2s/epoch - 3ms/step
Epoch 8/60
645/645 - 2s - loss: 4815.1968 - 2s/epoch - 4ms/step
Epoch 9/60
645/645 - 2s - loss: 4824.7476 - 2s/epoch - 3ms/step
Epoch 10/60
645/645 - 2s - loss: 4872.6567 - 2s/epoch - 3ms/step
Epoch 11/60
645/645 - 2s - loss: 4822.1362 - 2s/epoch - 2ms/step
Epoch 12/60
645/645 - 1s - loss: 4793.7754 - 1s/epoch - 2ms/step
Epoch 13/60
645/645 - 1s - loss: 4805.5146 - 1s/epoch - 2ms/step
Epoch 14/60
645/645 - 1s - loss: 4819.2993 - 1s/epoch - 2ms/step
Epoch 15/60
645/645 - 1s - loss: 4808.6509 - 1s/epoch - 2ms/step
Epoch 16/60
645/645 - 1s - loss: 4

In [21]:
# y_test

In [22]:
ann_pred = ann.predict(X_test)

4/4 [==============================] - 0s 1ms/step


In [23]:
from sklearn.metrics import r2_score

In [24]:
r2_score(y_test,ann_pred)

0.5660442669669602